# Metaclasses

Are they really needed? 99% no.

  “Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why).”
  — Tim Peters (Zen of Python author)

In [ ]:
# Ex1: Define a class dynamically

# Metaclass
Foo = type('Foo', (), {})
x = Foo()

print(x)

# Normal class
class Foo:
     pass

x = Foo()
print(x)


In [ ]:
# Ex2: Define a class with inheritance and add one attribute

# Metaclass
Bar = type('Bar', (Foo,), dict(attr=100))
x = Bar()

print(x.attr)
print(x.__class__)
print(x.__class__.__bases__)

# Normal class
class Bar(Foo):
    attr = 100

x = Bar()

print(x.attr)
print(x.__class__)
print(x.__class__.__bases__)

In [ ]:
# Ex3: Define a class with attributes and functions
# Metaclass
Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': lambda x : x.attr
    }
)

x = Foo()
print("Dinamically generated:")
print(x.attr)
print(x.attr_val())

# Normal class
class Foo:
    attr = 100
    def attr_val(self):
        return self.attr

print("------------------")
print("Statically defined:")
print(x.attr)
print(x.attr_val())

In [ ]:
# Class factory
class Meta(type):
    def __init__(
        cls, name, bases, dct
    ):
        cls.attr = 100
class X(metaclass=Meta):
    pass
X.attr

class Y(metaclass=Meta):
    pass
Y.attr

class Z(metaclass=Meta):
    pass
Z.attr

# Inheritance
class Base:
    attr = 100
class X(Base):
    pass
class Y(Base):
    pass
class Z(Base):
    pass


So, what about this 1%? When you need to create your code dynamically you will need to use metaclasses. Some examples:

- Do you depend on an external contract? (A contract seen as a microservice spec)
- Do you want to give a non programatic interface to define content schema?
- Do you want to enforce naming policy from a mix of legacy code and new code?
- Do you want to implement abstract classes without defining all required methods?

In [ ]:
# Example: Create classes from openapi contract
# RealWorld Conduit API Spec (demo.realworld.io)

# Requirements:
# pip install pyyaml requests
# On macOS, if you have problems with SSL certs: ln -s /etc/ssl/* <YOUR_PYTHON_PATH>/etc/openssl

try:
    from yaml import load, dump
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

# from urllib import request, parse
import json
# from urllib.error import HTTPError
import requests

# This will be our 'metaclass', to simplify, we will have a simple base class
# and will create dinamically a serie of derived classes
class APIComponent():
    api = 'https://api.realworld.io/api'
    headers = {'accept': 'application/json','Content-Type': 'application/json'}
    strict_mode = False # For this example we can accept that the spec is not strict
    def __init__(self,**kwargs):
        for attr in self.required: 
            if attr not in kwargs and self.strict_mode:
                raise KeyError(f"Required attribute {attr} not defined")
        for k,v in kwargs.items():
            if k in self.properties:
                setattr(self,k,v)
            elif self.strict_mode:
                raise KeyError(f"Invalid attribute {k}")

    def dict(self):
        response = dict()
        for k in self.properties:
            response[k] = getattr(self,k,None)
        return response
    
    def __str__(self):
        return json.dumps(self.dict(), default=lambda o: o.__dict__)

# Let's get the openapi spec
openapi_url = 'https://raw.githubusercontent.com/gothinkster/realworld/main/api/openapi.yml'
x = requests.get(openapi_url)
conduit_spec = load(x.text,Loader=Loader)

schemas = dict()

# We only create classes from the schemas, but to complete all the logic
# we would also create classes for paths and requests to know which object
# can use which path and complete the requests
for schema,content in conduit_spec['components']['schemas'].items():
    attributes = dict()
    attributes['required'] = content['required'] if 'required' in content else list()
    attributes['properties'] = content['properties'] if 'properties' in content else dict()

    schemas[schema] = type(schema,(APIComponent,),attributes)

print(f"We have created {len(schemas)} new dinamic classes")

# We get the classes that we need for this example
NewUser = schemas['NewUser']
LoginUser = schemas['LoginUser']
User = schemas['User']

# Declare a new user to create in the API, use your own values
my_data = {
    'username': 'testfxm',
    'password': 'testfxm',
    'email':    'testfxm@test.io'
}

In [ ]:
# TADAAA! This is the magic, we can create our NewUser instance from a dinamically created class
# From now on we could convert any response to their respective object and parse the values
new_user = NewUser(**my_data)

# This part should be encapsulated inside the class, but to simplify the example we will do it raw
data = dict(user=new_user.dict())
data = json.dumps(data).encode()

resp = requests.post(f"{new_user.api}/users", headers=new_user.headers, data=data)
parsed = resp.json()

print(parsed)
# We would need more logic to connect the rest of the components in the definition (LoginUserRequest, UserResponse...)
# to get the rest of the relations, but to simplify the example we will do it raw in the next cell

In [ ]:
# And now we will create the login request to obtain a token
login = LoginUser(**my_data)

# This part should be encapsulated inside a class, but to simplify the example we will do it raw
data = dict(user=login.dict())
data = json.dumps(data).encode()

resp = requests.post(f"{login.api}/users/login", headers=login.headers, data=data)
parsed = resp.json()

print(parsed)

# We create a new instance of my user with the information obtained
me = User(**parsed['user'])

# Observe that we loose 'token' value because it is not in the spec definition
print(me)